In [61]:
import gzip
from collections import defaultdict
import math
import scipy
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
import json
import time
import requests
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [4]:
#Get API KEY
file = open("mal_client_id.txt", 'r')
clientID = file.read()
file.close()

In [54]:
# let's generate a list of anime
# Methodology
# we generate according to the popularity list, rather than trolling through IDs, as not every ID is a valid entry.
# this has the added benefit of being able to cull the absolute most useless ones. 

# data
data = []

# constants
lowest_ranking_threshhold = 15000
entries_per_page = 500
fields = ["id", "title", "start_date", "end_date", "mean", "rank", "popularity", "num_list_users", "num_scoring_users", "nsfw", "genres", "status", "num_episodes", "start_season", "broadcast", "average_episode_duration", "rating", "studios"]
questionableFields = ["synopsis"] #probably going to include this still
apiURL = "https://api.myanimelist.net/v2/anime/ranking?"

head = {"X-MAL-CLIENT-ID": clientID}
par = {"ranking_type":"all", 
          "fields":"id,title,start_date,end_date,mean,rank,popularity,num_list_users,num_scoring_users,synopsis,nsfw,genres,status,num_episodes,start_season,broadcast,average_episode_duration,rating,studios", 
          "limit":entries_per_page,
          "offset":0,
}

response = requests.get(apiURL, headers=head, params=par)
infoArray = response.json()['data']
for i in infoArray:
    data.append(i['node'])
nextPage = response.json()['paging']['next']

#tentatively grabbing first 15000 anime, 500 at a time
for j in range(lowest_ranking_threshhold//entries_per_page):
    #getting data
    response = requests.get(nextPage, headers=head)
    infoArray = response.json()['data']
    for i in infoArray:
        data.append(i['node'])
    
    #now let's get the next page
    nextPage = response.json()['paging']['next']
    
    #to avoid getting ip banned, we wait for a bit
    #waiting is randomly between 1 and 3 seconds to simulate
    #some sort of natural traffic
    time.sleep(random.random()*2 + 1)
    print('We have done ' + str(j*500 + 500) + ' requests')


We have done 500 requests
We have done 1000 requests
We have done 1500 requests
We have done 2000 requests
We have done 2500 requests
We have done 3000 requests
We have done 3500 requests
We have done 4000 requests
We have done 4500 requests
We have done 5000 requests
We have done 5500 requests
We have done 6000 requests
We have done 6500 requests
We have done 7000 requests
We have done 7500 requests
We have done 8000 requests
We have done 8500 requests
We have done 9000 requests
We have done 9500 requests
We have done 10000 requests
We have done 10500 requests
We have done 11000 requests
We have done 11500 requests
We have done 12000 requests
We have done 12500 requests
We have done 13000 requests
We have done 13500 requests
We have done 14000 requests
We have done 14500 requests
We have done 15000 requests


In [57]:
#write to a file
anime_list = open("anime_list_details.json", 'w')
anime_list.write(json.dumps(data))
anime_list.close()

In [120]:
##########################################
## Getting user data                    ##
##########################################

In [123]:
#okay, now we want to just get a list of users, and we cannot do this through any API
#some experimenting, this gets the users between 1 and 122, which for all intents and purposes, is everyone
#the oldest recorded person is 122
baseURL = "https://myanimelist.net/users.php?"
para = {
    "cat":"user",
    "q":"",
    "loc":"",
    "agelow":"1",
    "agehigh":"122",
    "show":0,
}
numberOfUsernamesGoal = 100000

users = []

response = requests.get(baseURL, params=para)
#we keep going as long as we are getting okay status codes, we can always start again
while(response.status_code == 200 and len(users) < numberOfUsernamesGoal):
    foundUsernames = getUsernames(response.text)
    users += foundUsernames
    para['show'] += 24
    
    if(len(users) % 1000 < 24):
        print('Have retrieved ' + str(len(users)) + ' usernames')
    
    #to avoid getting ip banned, we wait for a bit
    #waiting is randomly between 1 and 3 seconds to simulate
    #some sort of natural traffic
    time.sleep(random.random()*2 + 1)
    
    #now let's make the request
    response = requests.get(baseURL, params=para)
    

Have retrieved 1008 usernames
Have retrieved 2016 usernames
Have retrieved 3000 usernames
Have retrieved 4008 usernames
Have retrieved 5015 usernames
Have retrieved 6023 usernames
Have retrieved 7007 usernames
Have retrieved 8015 usernames
Have retrieved 9023 usernames
Have retrieved 10007 usernames
Have retrieved 11015 usernames
Have retrieved 12023 usernames
Have retrieved 13007 usernames
Have retrieved 14015 usernames
Have retrieved 15023 usernames
Have retrieved 16007 usernames
Have retrieved 17015 usernames
Have retrieved 18023 usernames
Have retrieved 19007 usernames
Have retrieved 20015 usernames
Have retrieved 21023 usernames
Have retrieved 22007 usernames
Have retrieved 23015 usernames
Have retrieved 24023 usernames
Have retrieved 25007 usernames
Have retrieved 26015 usernames
Have retrieved 27023 usernames
Have retrieved 28007 usernames
Have retrieved 29015 usernames
Have retrieved 30023 usernames
Have retrieved 31007 usernames
Have retrieved 32015 usernames
Have retrieved 33

In [124]:
len(users)

100006

In [127]:
#write to a file
user_list = open("username_list.txt", 'w')
user_list.write(json.dumps(users))
user_list.close()

In [128]:
# Getting details on user's animelist

In [132]:
# first, we get our users
user_file = open("username_list.txt", 'r')
usernames = json.load(user_file)

In [268]:
data = []
count = 0 #this is here so we can keep track of how many usernames we've gone through and keep going in case of an error

In [298]:
# This should be just an array of usernames
# for each username, we will enter it into our data dict

baseURL = "https://api.myanimelist.net/v2/users/"
appendURL = "/animelist?"
para = {
    "fields":"list_status,start_date,end_date,id",
    "limit":1000,
    "offset":0,
}



for username in usernames[count-1:]:
    count += 1
    response = requests.get(baseURL+username+appendURL, params=para, headers=head)    
    
    #if user has a private list or there is some other error accessing the list, we skip
    if('error' in response.json()):
        continue
        
    response_data = response.json()['data']

        
    #sanity check print statements
    if(count % 1000 == 1):
        print('Have retrieved animelists for ' + str(count) + ' users')
        
    #if something happens let's stop
    if(response.status_code != 200):
        print('Got HTTP status code: ' + str(response.status_code) + ', exiting now.')
        break
    
    #to avoid getting ip banned or rate limited, we wait 270ms
    time.sleep(0.27)
    
    #storing JSON is too big. let's put it in a csv.
    #we are really just storing the list as an array with csv values
    #user,id,start_date,end_date,score,num_episodes_watched,status
    #status encoding: watching = 0, completed = 1, on_hold = 2, dropped = 3, plan_to_watch = 4

    for entry in response_data:
        data.append(str(username) + ',' + str(entry['node']['id']) + ',' + str(getCSVDates(entry['list_status'])) + ',' + str(entry['list_status']['score']) + ',' + str(entry['list_status']['num_episodes_watched']) + ',' + str(getStatusEncoding(entry['list_status'].get('status'))))
    
    #if there are more than 1000 entries, we need to keep going through pages
    while('next' in response.json()['paging']):
        response = requests.get(response.json()['paging']['next'], headers=head)
        response_data = response.json()['data']
        for entry in response_data:
            data.append(str(username) + ',' + str(entry['node']['id']) + ',' + str(getCSVDates(entry['list_status'])) + ',' + str(entry['list_status']['score']) + ',' + str(entry['list_status']['num_episodes_watched']) + ',' + str(getStatusEncoding(entry['list_status'].get('status'))))
        time.sleep(0.2)

Have retrieved animelists for 51001 users


JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>403 ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
Request blocked.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>
Generated by cloudfront (CloudFront)
Request ID: 1NTPLxFZzcxqP5sTiwgJ2xmHChBoJyhjuuQHhB2sGizyxP-XQIEGRA==
</PRE>
<ADDRESS>
</ADDRESS>
</BODY></HTML>: 0

In [ ]:
count

In [ ]:
#start date and end date could be absent from the list status.
#this function returns strings correctly for the csv for both of them
def getCSVDates(list_status):
    start_date_csv = ""
    if('start_date' not in list_status):
        start_date_csv = ","
    else:
        start_date_csv = list_status['start_date'] + ','
        
    end_date_csv = ""
    if('finish_date' not in list_status):
        end_date_csv = ""
    else:
        end_date_csv = list_status['finish_date'] 
        
    return start_date_csv + end_date_csv

In [ ]:
def getStatusEncoding(status):
    if status == 'watching':
        return 0
    if status == 'completed':
        return 1
    if status == 'on_hold':
        return 2
    if status == 'dropped':
        return 3
    if status == 'plan_to_watch':
        return 4
    return 5

In [300]:
len(data)

8762677

In [305]:
#write to a file
user_list = open("user_animelist_details.txt", 'w',encoding='utf-8')
for i in data:
    user_list.write(i+'\n')
user_list.close()